In [ ]:
import pandas as pd

melb_data = pd.read_csv('/Users/MacBook/Desktop/Data Saintist/IDE/PY_11_Работа с Pandas/data_csv/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

if __name__ == '__main__':
    print(melb_data.head())  

Лайфхак. Старайтесь всегда оставлять переменную с первоначальным DataFrame неизменной, создавайте копию исходной таблицы и совершайте преобразования на ней. Это оградит вас от ошибок, которые можно совершить при подготовке данных. Например, если вы понимаете, что преобразование оказалось неудачным, достаточно будет лишь запустить ячейку, в которой вы производите копирование, а не читать таблицу заново. Особенно критичным это может быть, когда количество строк в таблице исчисляется миллионами и её чтение занимает до нескольких минут.

Среди списка базовых операций над столбцами в Pandas важное место занимает возможность удаления столбцов из таблицы. Это может быть полезно, например, когда в данных есть признаки, которые не несут полезной информации.
За удаление строк и столбцов в таблице отвечает метод drop().
    labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
    axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
    inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

In [ ]:
import pandas as pd

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

Pandas поддерживает базовые математические операции между столбцами: столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

In [ ]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

А теперь введём признак MeanRoomsSquare, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:

In [ ]:
melb_df['MeanRoomSquare']=melb_df['BuildingArea']/total_rooms
display(melb_df['MeanRoomSquare'])

Можно ввести ещё один интересный признак — AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму:
Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

        Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.
        Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.
        Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

In [ ]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])


Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой. Для этого был создан единый способ обозначения даты и времени. 
Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

In [ ]:
display(melb_df['Date'])

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

        date — дата;
        year, month, day — год, месяц, день;
        time — время;
        hour, minute, second — час, минута, секунда;

        dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
        day_name — название дня недели;
        dayofyear — порядковый день года;
        quarter — квартал (интервал в три месяца).



In [ ]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:
В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

In [ ]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [ ]:
display(delta_days.dt.days)

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [ ]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

melb_df = melb_df.drop('YearBuilt', axis=1)